The Network IOP enables raw access to the Ethernet interface from within Python.  The usage is similar in many ways to sending and receiving ethernet frames using raw sockets.  One advantage of this access is that packets can be sent with low-latency, bypassing the normal linux kernel stack.  Another advantage is that access to the network interface is memory-mapped, enabling network-connected accelerators to be prototyped running on the ARM cores and then migrated into the programmable logic.

In software, the code typically executes with about 10ms between publish events.
When accelerated using the FPGA, the code executes about 120us between publish events.

In [ ]:
// AXILiteS
// 0x00 : Control signals
//        bit 0  - ap_start (Read/Write/COH)
//        bit 1  - ap_done (Read/COR)
//        bit 2  - ap_idle (Read)
//        bit 3  - ap_ready (Read)
//        bit 7  - auto_restart (Read/Write)
//        others - reserved
// 0x04 : Global Interrupt Enable Register
//        bit 0  - Global Interrupt Enable (Read/Write)
//        others - reserved
// 0x08 : IP Interrupt Enable Register (Read/Write)
//        bit 0  - Channel 0 (ap_done)
//        bit 1  - Channel 1 (ap_ready)
//        others - reserved
// 0x0c : IP Interrupt Status Register (Read/TOW)
//        bit 0  - Channel 0 (ap_done)
//        bit 1  - Channel 1 (ap_ready)
//        others - reserved
// 0x10 : Data signal of b
//        bit 0  - b[0] (Read/Write)
//        others - reserved
// 0x14 : reserved
// 0x18 : Data signal of macAddress_V
//        bit 31~0 - macAddress_V[31:0] (Read/Write)
// 0x1c : Data signal of macAddress_V
//        bit 15~0 - macAddress_V[47:32] (Read/Write)
//        others   - reserved
// 0x20 : reserved
// 0x24 : Data signal of ipAddress_V
//        bit 31~0 - ipAddress_V[31:0] (Read/Write)
// 0x28 : reserved
// 0x2c : Data signal of i
//        bit 31~0 - i[31:0] (Read/Write)
// 0x30 : reserved
// 0x34 : Data signal of destIP_V
//        bit 31~0 - destIP_V[31:0] (Read/Write)
// 0x38 : reserved
// 0x3c : Data signal of destPort
//        bit 31~0 - destPort[31:0] (Read/Write)
// 0x40 : reserved
// 0x44 : Data signal of topicID_V
//        bit 15~0 - topicID_V[15:0] (Read/Write)
//        others   - reserved
// 0x48 : reserved
// 0x4c : Data signal of qos
//        bit 31~0 - qos[31:0] (Read/Write)
// 0x50 : reserved
// 0x54 : Data signal of message
//        bit 31~0 - message[31:0] (Read/Write)
// 0x58 : reserved
// 0x5c : Data signal of validMessage
//        bit 0  - validMessage[0] (Read/Write)
//        others - reserved
// 0x60 : reserved
// 0x64 : Data signal of networkIOP_offset
//        bit 31~0 - networkIOP_offset[31:0] (Read/Write)
// 0x68 : reserved
// 0x6c : Data signal of count
//        bit 31~0 - count[31:0] (Read/Write)
// 0x70 : reserved
// 0x74 : Data signal of size
//        bit 31~0 - size[31:0] (Read/Write)
// 0x78 : reserved
// 0x7c : Data signal of reset
//        bit 0  - reset[0] (Read/Write)
//        others - reserved
// 0x80 : reserved
// 0x84 : Data signal of p_verbose
//        bit 0  - p_verbose[0] (Read/Write)
//        others - reserved
// 0x88 : reserved
// 0x8c : Data signal of events_completed
//        bit 31~0 - events_completed[31:0] (Read)
// 0x90 : Control signal of events_completed
//        bit 0  - events_completed_ap_vld (Read/COR)
//        others - reserved
// 0x94 : Data signal of publishes_sent
//        bit 31~0 - publishes_sent[31:0] (Read)
// 0x98 : Control signal of publishes_sent
//        bit 0  - publishes_sent_ap_vld (Read/COR)
//        others - reserved
// 0x9c : Data signal of packets_received
//        bit 31~0 - packets_received[31:0] (Read)
// 0xa0 : Control signal of packets_received
//        bit 0  - packets_received_ap_vld (Read/COR)
//        others - reserved
// 0xa4 : Data signal of packets_sent
//        bit 31~0 - packets_sent[31:0] (Read)
// 0xa8 : Control signal of packets_sent
//        bit 0  - packets_sent_ap_vld (Read/COR)
//        others - reserved
// (SC = Self Clear, COR = Clear on Read, TOW = Toggle on Write, COH = Clear on Handshake)


In [1]:
print("Importing scapy...")
from scapy.all import *
exec(open("MQTTSN.py").read())

from cffi import FFI
from pynq import Overlay
from pynq import MMIO
from wurlitzer import sys_pipes

myIPStr = '192.168.1.104'
myIP = struct.unpack(">L", inet_aton(myIPStr))[0]
myMACStr = '8a:70:bd:29:2b:40'
myMAC = struct.unpack(">Q", b'\0\0'+mac2str(myMACStr))[0]

class FFIOverlay(Overlay):
    def __init__(self, bitfile_name, interface_string, dll_name=None):
        """Construct a new overlay managed through FFI.
        bitfile_name -- The filename of a bitstream
        interface_string -- The FFI interface description of the overlay (a string containing c-like prototypes)
        dll_name -- Optional filename of the dll to be loaded.  If not specified, then assume the dll filename is
                derived from bitfile_name by dropping the .bit extension"""
        super().__init__(bitfile_name)
        self.interface_string = interface_string
        self.dll_name = dll_name if dll_name else os.path.splitext(bitfile_name)[0]
        self.dll = None
        
    def download(self):
        """Download the overlay and initialize the corresponding FFI interface."""
        super().download()
        self.ffi = FFI()
        self.ffi.cdef(self.interface_string)
        with sys_pipes():
            self.dll = self.ffi.dlopen(self.dll_name)

class ReadSensorOverlay(FFIOverlay):
    """An overlay for accessing the networkIOP"""
    def __init__(self, accelerated=True):
        overlay = "pynq-sdsoc_qos_direct/liblibsw-mqttsn-publish.so.bit"
        if(accelerated):
            overlay = "pynq-sdsoc_qos_direct_noadapter/libread_and_process_packet-mqttsn-publish.so.bit"
        super().__init__(
            overlay,    
            """
void init_ethernet_raw(const char *interface, uint16_t myPort);
float read_sensor(volatile char * sensor);
void Top(int size, int count,
         unsigned long long macAddress,
         unsigned int ipAddress,
         unsigned int destIP,
         int destPort,
         unsigned short topicID,
         int qos,
         bool verbose,         
         volatile char * networkIOP,
         volatile char * sensor);
void sds_mmap(int phys, int length, void *virtual);
void *sds_alloc(size_t size);
""")
        self.TMP2_ptr = None
        
    def read_sensor(self, TMP2):
        """Read the value of a temperature sensor."""
        if(self.TMP2_ptr is None):
            self.TMP2_ptr = self.map(TMP2.mmio)
        return ol.dll.read_sensor(self.TMP2_ptr)
    
    def init_ethernet_raw(self, interface, port):
        with sys_pipes():
            ol.dll.init_ethernet_raw(interface.encode('utf-8'),1884)

    def map(self, mmio):
        """Map the given mmio interface in a way that SDSoc can use it."""
        virtaddr = ol.ffi.from_buffer(mmio.mem)
        #print(hex(mmio.base_addr +mmio.virt_offset), "->", virtaddr)
        ol.dll.sds_mmap(mmio.base_addr+mmio.virt_offset, mmio.length, virtaddr)
        return virtaddr

    def Top(self, size, count, interface, destIP, destPort, topicID, qos, verbose, netIOP, TMP2):
        macAddress2 = struct.unpack(">Q",b'\0\0'+get_if_raw_hwaddr(interface))[0]
        ipAddress2 = myIP
        destIP2 = struct.unpack(">L",inet_aton(destIP))[0]
        print(ipAddress2, ",", macAddress2)
        macAddressArg = self.ffi.cast("unsigned long long", macAddress2)
        ipAddressArg = self.ffi.cast("unsigned int", ipAddress2)
        destIPArg = self.ffi.cast("unsigned int", destIP2)
        with sys_pipes():
            ol.dll.Top(size, count, macAddressArg, ipAddressArg, destIPArg, destPort, topicID, qos, verbose, 
                       ol.ffi.from_buffer(netIOP._mmio.mem),
                       ol.ffi.from_buffer(TMP2.mmio.mem))

    def Top2(self, size, count, macAddress, ipAddress, destIP, destPort, topicID, qos, verbose, netIOP, TMP2):
        """Publish data from the given temperature sensor to an MQTTSN server.
        size -- The size of frames to generate
        count -- The number of publish events to complete
        macAddress -- The MAC Address of the PL accelerator (not the host MAC address)
        ipAddress -- The IP Address of the PL accelerator (not the host IP address)
        destIP -- The IP Address of the MQTTSN server
        topicID -- The topic ID to publish on
        qos -- The MQTTSN qos to use
        verbose -- non-zero to get verbose debugging information
        netIOP -- The network IOP object
        TMP2 -- The temperature sensor object
        """
        macAddress2 = macAddress
        ipAddress2 = ipAddress
        destIP2 = struct.unpack(">L",inet_aton(destIP))[0]
        macAddressArg = self.ffi.cast("unsigned long long", macAddress2)
        ipAddressArg = self.ffi.cast("unsigned int", ipAddress2)
        destIPArg = self.ffi.cast("unsigned int", destIP2)
        with sys_pipes():
            netIOP_ptr = self.map(netIOP._mmio)
            TMP2_ptr = self.map(TMP2.mmio)
            ol.dll.Top(size, count, macAddressArg, ipAddressArg, destIPArg, destPort, topicID, qos, verbose, 
                       netIOP_ptr,
                       TMP2_ptr)
            
            
    def Top3(self, size, count, macAddress, ipAddress, destIP, destPort, topicID, qos, verbose, netIOP, TMP2):
        macAddress2 = macAddress
        ipAddress2 = ipAddress
        destIP2 = struct.unpack(">L",inet_aton(destIP))[0]
        macAddressArg = self.ffi.cast("unsigned long long", macAddress2)
        ipAddressArg = self.ffi.cast("unsigned int", ipAddress2)
        destIPArg = self.ffi.cast("unsigned int", destIP2)
        netIOP_ptr = self.map(netIOP._mmio)
        netIOP_phys = netIOP._mmio.base_addr+netIOP._mmio.virt_offset
        TMP2_ptr = self.map(TMP2.mmio)
        print(hex(destIP2))
        acc_mmio = MMIO(0x83c00000, 0x10000);
        acc_mmio.write(0x10, 1) # Write b
        acc_mmio.write(0x18, macAddress & 0xFFFFFFFF)
        acc_mmio.write(0x1c, macAddress >> 32 + 1)
        acc_mmio.write(0x24, ipAddress)
        acc_mmio.write(0x2c, 1)
        acc_mmio.write(0x34, destIP2)
        acc_mmio.write(0x3c, destPort)
        acc_mmio.write(0x44, topicID)
        acc_mmio.write(0x4c, qos)
        acc_mmio.write(0x54, 0x0)
        acc_mmio.write(0x5c, 1) # validMessage
        acc_mmio.write(0x64, netIOP_phys)
        acc_mmio.write(0x6c, count)
        acc_mmio.write(0x74, size)
        acc_mmio.write(0x7c, 1) # reset
        acc_mmio.write(0x84, verbose) # verbose
        
        #Execute the accelerator once to reset things
        acc_mmio.write(0x0, 1) # start
        status = acc_mmio.read(0x0)
        while(status&0x2 == 0):
            status = acc_mmio.read(0x0)       
        
        acc_mmio.write(0x7c, 0) # reset
           
        #for i in range(0,0x8c,0x4):
        #    print(hex(i),hex(acc_mmio.read(i)))
        #for i in range(0,0x800,0x4):
        #    t = netIOP._mmio.read(i)
        #    if(t != 0):
        #        print(hex(i),hex(t))
        #print("status",acc_mmio.read(0x0)) # getStatus
        eventsCompleted = 0
        i = 0
        while eventsCompleted < count:
            #Wait for accelerator to be idle.
            status = acc_mmio.read(0x0)
            while(status&0x4 == 0):
                status = acc_mmio.read(0x0)
            #Set our inputs and start.
            acc_mmio.write(0x2c, i)
            acc_mmio.write(0x5c, i%2) # validMessage
            acc_mmio.write(0x0, 1) # start
            i = i+1
            eventsCompleted = acc_mmio.read(0x8c)            
            if(i%1000 == 0):
                print("status", status)
                print("EventsCompleted:", eventsCompleted)
                print("PublishesSent:", acc_mmio.read(0x94))
        #print("status",status) # getStatus       
        print("calls", i)
        print("EventsCompleted:", eventsCompleted)
        print("PublishesSent:", acc_mmio.read(0x94))
        print("PacketsReceived:", acc_mmio.read(0x9c))
        print("PacketsSent:", acc_mmio.read(0xa4))
            
ol = ReadSensorOverlay()    
ol.download()

from board.slurper import *

class L2PynqSocket(SuperSocket):
    """A scapy-like socket object that reads and writes packets using the Pynq NetworkIOP interface"""
    desc = "read/write packets at layer 2 using Pynq bypass"
    __slurper = PacketSlurper()
    def __init__(self, iface = None, type = ETH_P_ALL, filter=None, nofilter=0):
        if iface is None:
            iface = conf.iface
        self.LL = Ether
        self.slurper = L2PynqSocket.__slurper
        
    def flush(self):
        """Flush any packets buffered up in the interface.
        Packets are buffered up in the interface.  Since packets can potentially come in faster than being read,
        this causes a problem since the buffer of packets often starts close to full and can drop newer packets
        while keeping older stale packets around.  This method enables an application to flush stale packets from
        the buffer before starting a packet exchange to avoid losing packets of interest."""    
        i = 0
        while(self.has_packet()):
            i=i+1
            self.recv()
        print(i, "packets flushed")    
    def has_packet(self):
        """Return true if the interface has a packet to read."""
        return self.slurper.has_packet()
    
    def srp1(self, outframe, valid_ack, ptype):
        """Send the given outframe and wait for a valid acknolwedgement of the given ptype as determined by the valid_ack function.
        valid_ack must model (frame, ptype) -> bool.  This function blocks until a valid acknowledgement is received."""
        self.send(outframe)
        frame = None
        while(not frame):
            if(self.has_packet()):
                frame = self.recv()
                if(not valid_ack(frame, ptype)):
                    frame = None
        return frame
        
    def recv(self, x=MTU):
        """Receive a frame.  This function blocks until a frame is received."""
        pkt = self.slurper.recv()
        try:
            q = self.LL(pkt)
        except KeyboardInterrupt:
            raise
        except:
            if conf.debug_dissector:
                raise
            q = conf.raw_layer(pkt)
        q.time = time.time()
        return q
    
    def send(self, x):
        """Send a frame.  This function blocks until the frame has been sent."""
        if hasattr(x, "sent_time"):
            x.sent_time = time.time()
        return self.slurper.send(bytes(x))        

conf.L2PynqSocket = L2PynqSocket

from pynq.iop import PMODB
from pynq.iop import Pmod_TMP2

import timeit

mytmp = Pmod_TMP2(PMODB)
# Read the temperature sensor
print(mytmp.read())
# Read the temperature sensor again, through C code.
with sys_pipes():
    print(ol.read_sensor(mytmp))

Importing scapy...


30.2
30.1875


In [ ]:
from pynq import Overlay
from pynq.iop import Pmod_OLED
from pynq.iop import Pmod_TMP2
from pynq.iop import PMODB
from pynq.iop import PMODA
from board.bram import BRAM as netIOP
import timeit
import time

serverIP = "192.168.3.99"
serverPort = 1884

mynet = netIOP()
#pmod_oled = Pmod_OLED(PMODA)
mytmp = Pmod_TMP2(PMODB)
#ol.Top3(100, 1000, myMAC, myIP, serverIP, serverPort, 1, 0, 1, mynet, mytmp)
conf.L2PynqSocket().flush()
ol.Top3(100, 5, myMAC, myIP, serverIP, serverPort, 1, 0, 1, mynet, mytmp)


`
xilinx@pynq:~$ sudo ifconfig br0:0 192.168.1.99
xilinx@pynq:~$ sudo ifconfig br0:1 192.168.3.99
xilinx@pynq:~$ ./pynq_enet_kernel/kernel/link_up.sh
`

In [2]:
def mqttsn_valid_ack(ack,t):
    """
    Return True if ack is a valid acknowledgement packet of type t.
    In addition, handle ARP request packets by sending an ARP reply.
    """
    #print("ether:", ack.summary())
    if(ARP in ack):
        arp = ack[ARP]
        if(arp.pdst == myIPStr):
            print("ARP ME")
            arpreply = Ether(dst=ack.src, src=myMACStr)/ARP(op='is-at', psrc= myIPStr, hwsrc=myMACStr, pdst=arp.psrc, hwdst=arp.hwdst)
            conf.L2PynqSocket().send(arpreply)
    if(IP not in ack):
        #print("not IP")
        return False
    if(IP in ack and isinstance(ack[IP].payload, ICMP)):
        print("Error response:")
        #ack[IP].payload.show()
        return False
    if(MQTTSN in ack):
        print("MQTTSN:", ack.summary())
        # FIXME: model this in the same way that scapy does in the packet description.
        if(isinstance(ack[MQTTSN].payload, t)):
            return True
        else:
            print("Unexpected response should have been "+ str(t)+":")
            ack.payload.show()
            return False
    return False

class MQTT_client:
    def __init__(self, serverIP, serverPort, name, verbose=0):
        """
        Create a new client object representing a connection to an MQTTSN server.
        serverIP -- a dotted string representing the IP of the server
        serverPort -- an integer represeting the port of the server (usually 1884)
        name -- the name of the client
        verbose -- optional.  If non-zero, then get verbose debugging feedback about the connection.
        """
        self.serverIP = serverIP
        self.serverPort = serverPort
        self.client = name
        self.verbose = verbose
        self.srcIP = myIP
        self.srcMAC = myMAC
        self.frame = None
        self.socket = conf.L2PynqSocket()
    
    def __enter__(self):
        self.connect()
        
        # Fixme: on failure throw exception?
        return self
    
    def __exit__(self, type, value, traceback):
        self.disconnect()
        
    def connect(self):
        """
        Connect to the server.  This blocks until an acknowledgement is received.
        """
        frame = Ether(src=myMACStr, dst='FF:FF:FF:FF:FF:FF')/IP(src=myIPStr, dst=self.serverIP)/UDP(sport=50000,dport=self.serverPort)/MQTTSN()/MQTTSN_CONNECT(client=self.client)
        connack_frame = self.socket.srp1(frame, mqttsn_valid_ack, MQTTSN_CONNACK)
        print("connect done")
        return True

    def disconnect(self):
        """
        Disconnect from the server.
        """
        pass  # FIXME: implement this
        #disconnack = send(IP(dst=serverIP)/UDP(sport=50000,dport=serverPort)/MQTTSN()/MQTTSN_DISCONNECT(), verbose=self.verbose)
        # rsmb seems to respond with out the disconnect payload.
        # return valid_ack(disconnack,MQTTSN_DISCONNECT)
    
    def register(self, topic):
        """
        Register the given topic with the server.  This blocks until an acknowledgement is received.
        Return the topicID that should be used to publish on the given topic
        """
        frame = Ether(src=myMACStr, dst='FF:FF:FF:FF:FF:FF')/IP(src=myIPStr, dst=self.serverIP)/UDP(sport=50000,dport=serverPort)/MQTTSN()/MQTTSN_REGISTER(topic=topic)
        regack_frame = self.socket.srp1(frame, mqttsn_valid_ack, MQTTSN_REGACK)
        print("register done")
        return regack_frame[MQTTSN_REGACK].topicID

    def publish(self, topicID, message, qos=1):
        """
        Publish the given message on the topic associated with the given topicID to the server.
        This blocks until an acknowledgement is received.
        Return True if the publish succeeds
        """        
        # Publish on the topic again with qos=1 (guaraunteed delivery).
        self.frame = bytes(Ether(src=myMACStr, dst='FF:FF:FF:FF:FF:FF')/IP(src=myIPStr, dst=self.serverIP)/UDP(sport=50000,dport=serverPort)/MQTTSN()/MQTTSN_PUBLISH(qos=qos, topicID=topicID, message=message))
        if(qos == 0):
            self.socket.send(self.frame)
        if(qos == 1):
            puback_frame = self.socket.srp1(self.frame, mqttsn_valid_ack, MQTTSN_PUBACK)
        return True
   
    def publish_temp(self, netIOP, sensor, topicID, qos, range):
        ol.Top3(100, len(range), myMAC, myIP, self.serverIP, self.serverPort, topicID, qos, self.verbose, netIOP, sensor)
        # Publish on the topic again with qos=1 (guaraunteed delivery).
        #puback = sr1(IP(dst=serverIP)/UDP(sport=50000,dport=serverPort)/MQTTSN()/MQTTSN_PUBLISH(qos=1, topicID=topicID, message=message), verbose=self.verbose)
        #if not valid_ack(puback,MQTTSN_PUBACK):
        #    return False
        return True


In [3]:
from pynq import Overlay
from pynq.iop import Pmod_OLED
from pynq.iop import Pmod_TMP2
from pynq.iop import PMODB
from pynq.iop import PMODA
from board.bram import BRAM as netIOP
import timeit
import time

ol.init_ethernet_raw("br0",1884)

serverIP = "192.168.3.99"
serverPort = 1884

mynet = netIOP()
#pmod_oled = Pmod_OLED(PMODA)
mytmp = Pmod_TMP2(PMODB)

conf.L2PynqSocket().flush()
count = 4000
with sys_pipes():
  with MQTT_client(serverIP, serverPort, "client-temp-accelerator", verbose=1) as client:
    topicID = client.register("temp")
    client.publish(topicID,"test")
    conf.L2PynqSocket().flush()
    start_time = timeit.default_timer()
    client.publish_temp(mynet, mytmp, topicID, 0, range(0,count))
    elapsed = timeit.default_timer() - start_time
    print(str(elapsed)+" seconds.")
    print(str(count/elapsed)+" messages/second.")
    

ifindex = 4
Bound to 0x0:0x18:0x3e:0x2:0x48:0x3d-172.19.73.242
139 packets flushed
MQTTSN: Ether / IP / UDP 192.168.3.99:1884 > 192.168.1.104:50000 / MQTTSN / MQTTSN_CONNACK / Padding
connect done
MQTTSN: Ether / IP / UDP 192.168.3.99:1884 > 192.168.1.104:50000 / MQTTSN / MQTTSN_REGACK / Padding
register done
MQTTSN: Ether / IP / UDP 192.168.3.99:1884 > 192.168.1.104:50000 / MQTTSN / MQTTSN_PUBACK / Padding
1 packets flushed
0xc0a80363
status 6
EventsCompleted: 499
PublishesSent: 499
status 6
EventsCompleted: 999
PublishesSent: 999
status 6
EventsCompleted: 1499
PublishesSent: 1499
status 6
EventsCompleted: 1999
PublishesSent: 1999
status 6
EventsCompleted: 2498
PublishesSent: 2498
status 6
EventsCompleted: 2996
PublishesSent: 2996
status 6
EventsCompleted: 3494
PublishesSent: 3494
status 6
EventsCompleted: 3994
PublishesSent: 3994
calls 8012
EventsCompleted: 4000
PublishesSent: 4000
PacketsReceived: 19
PacketsSent: 4000
1.2116258109999762 seconds.
3301.3492810117086 messages/second.


In [ ]:
from pynq import Overlay
from pynq.iop import Pmod_OLED
from pynq.iop import Pmod_TMP2
from pynq.iop import PMODB
from pynq.iop import PMODA
import timeit

#ol = Overlay("base.bit")
#ol.download()

#serverIP = "192.168.1.103"
#serverPort = 1884

pmod_oled = Pmod_OLED(PMODA)
mytmp = Pmod_TMP2(PMODB)

conf.L2PynqSocket().flush()
count = 400
with sys_pipes():
  with MQTT_client(serverIP, serverPort, "client-temp") as client:
    temp_topicID = client.register("temp")
    client.publish(topicID,"test")
    start_time = timeit.default_timer()
    conf.L2PynqSocket().flush()    
    temperature = mytmp.read()
    tempStr = str(temperature)
    for i in range(0,count):
        #pmod_oled.clear()
        #pmod_oled.write(tempStr)
        client.publish(temp_topicID, tempStr, qos=0)
        
    elapsed = timeit.default_timer() - start_time
    print(str(elapsed)+" seconds.")
    print(str(count/elapsed)+" messages/second.")
    

In [ ]:
frame = conf.L2PynqSocket().recv()
print(frame)

In [ ]:
#Measure outgoing packet rate
import timeit
import numpy as np
sizes = [64, 128, 256, 512, 1024, 1500] #range(11,1520,100)
pps = []
bps = []
usperpacket = []
cyclesperword = []
theoretical = []
from pynq import PL
from pynq import MMIO
mmio = MMIO(0xFFFC0000, 0x10000)
for size in sizes:
    payload=b''.join([b'0' for i in range(0,size)])
    frame = Ether(src=myMACStr, dst='FF:FF:FF:FF:FF:FF')/IP(src=myIPStr, dst="192.168.1.2")/UDP(sport=50000, dport=1884)/MQTTSN()/MQTTSN_CONNECT()
    #frame = bytes(frame)+payload
    #frame = payload
    count = 100
    slurper = conf.L2PynqSocket().slurper
    scapySocket = conf.L2socket()
    write32 = slurper.write32
    array = mmio.array
    mem = mmio.mem
    array = slurper._mmio.array
    mem = slurper._mmio.mem
    leng = len(frame)
    start_time = timeit.default_timer()
    for i in range(0,count):
        frame2=bytes(frame)
        #scapySocket.send(frame);
        #write32(slurper.TX_CTRL_OFFSET, 0xa0000000)
        #write32(slurper.TX_CTRL_OFFSET+1, 0x02)
        #write32(slurper.TX_DATA_OFFSET, frame)
        #write32(slurper.TX_LEN_OFFSET, leng)
        #write32(slurper.TX_EN_OFFSET, 0x01)
        #num_words = leng >> 2 + 1
        #buf = np.frombuffer(frame, np.uint32, num_words, 0)
        #array[0:num_words] = buf
        #array[slurper.TX_LEN_OFFSET] = leng
        #array[slurper.TX_EN_OFFSET] = 0x01
        #mem.seek(0x0)
        #mem.write(frame)
        #array[0x194] = leng
        #array[0x190] = 0x01
        slurper.send(frame2)
    #frame=slurper.recv()
    elapsed = timeit.default_timer() - start_time
    bps.append(count*len(frame)*8/elapsed/1000000)
    pps.append(count/elapsed)
    usperpacket.append(1000000/(count/elapsed))
    cyclesperword.append((100000000*elapsed)/(count*(len(frame)/4)))
    theoretical.append(100000000/(len(frame)/4))
    #print(str(elapsed)+" seconds.")
    #print(str(count/elapsed)+" messages/second.")
    #print(len(frame))
    #print(str(count*len(frame)*8/elapsed) + " bits/second.")

print(pps)
print(theoretical)
print(usperpacket, " microseconds per packet")
print(cyclesperword, " cycles per word")
import matplotlib.pyplot as plt

plt.title("Achieved bits per second")
plt.plot(sizes, bps)
plt.show()
plt.title("Achieved packets per second")
plt.plot(sizes, pps)
plt.show()

In [ ]:
print(frame)

In [ ]:
# Test how fast we can read the temperature sensor
count = 1000
start_time = timeit.default_timer()
for i in range(0,count):
    #x = ol.read_sensor(mytmp)
    x = mytmp.read()
    
elapsed = timeit.default_timer() - start_time
print(str(elapsed)+" seconds.")
print(str(count/elapsed)+" reads/second.")
print(str(x)+" degrees.")